In [30]:
####Imports and Setup
import os
import sys
import time
import glob
import datetime
import sqlite3
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import sklearn
import patsy
import numpy as np
import pandas as pd
from scipy import stats
import seaborn as sns
from sklearn import datasets, linear_model
from sklearn.model_selection import train_test_split
###########################
import numpy as np
import scipy 
from scipy.interpolate import interp1d
import seaborn as sns
import pandas as pd
from sklearn.preprocessing import StandardScaler
# We are using the BernoulliNB version of Naive Bayes, which assumes predictors are binary encoded.
from sklearn.naive_bayes import BernoulliNB
from sklearn.cross_validation import cross_val_score, train_test_split

from sklearn.feature_extraction.text import CountVectorizer

from sklearn.linear_model import LogisticRegression

from sklearn.linear_model import Ridge, Lasso, ElasticNet, LinearRegression, RidgeCV, LassoCV, ElasticNetCV
from sklearn.cross_validation import cross_val_score

from sklearn.linear_model import LogisticRegressionCV

%config InlineBackend.figure_format = 'retina'
%matplotlib inline

In [31]:
def correlation_heat_map(df):
    corrs = df.corr()

    # Set the default matplotlib figure size:
    fig, ax = plt.subplots(figsize=(12,12))

    # Generate a mask for the upper triangle (taken from seaborn example gallery)
    mask = np.zeros_like(corrs, dtype=np.bool)
    mask[np.triu_indices_from(mask)] = True

    # Plot the heatmap with seaborn.
    # Assign the matplotlib axis the function returns. This will let us resize the labels.
    ax = sns.heatmap(corrs, mask=mask, annot=True)

    # Resize the labels.
    ax.set_xticklabels(ax.xaxis.get_ticklabels(), fontsize=14, rotation=30)
    ax.set_yticklabels(ax.yaxis.get_ticklabels(), fontsize=14, rotation=0)

    # If you put plt.show() at the bottom, it prevents those useless printouts from matplotlib.
    plt.show()

def update_columns(col_names, label):
    new_names = []
    new_names.append(col_names[0])
    for name in col_names[1:]:
        name = label + name
        new_names.append(name)
    return new_names

def is_ascii(column):
    column_list = []
    for col in column:
        for letter in col:
            if ord(letter) > 128:
                column_list.append(col)
                continue
    return column_list

def del_cols(df):
    cols = list(df.columns.values)
    cols_del = []
    for x in cols:
        for ch in x:
            if ch not in 'abcdefghijklmnopqrstuvwxyz_0123456789-':
                cols_del.append(x)
                continue
    return cols_del

In [32]:
conn_addf_metadata = sqlite3.connect('../../Capstone/MillionSongSubset/AdditionalFiles/subset_track_metadata.db')
conn_lyric = sqlite3.connect('../../Capstone/mxm_dataset.db')

In [33]:
#Combined data set on lyrics and genres (no sub-genres in the data set)
df_lyric_comb = pd.read_csv('../Capestone/df_lyric_genre_metadata.csv')

#Combined data with sub-genres (no lyrics in the data set)
df_tags_comb = pd.read_csv('../Capestone/df_sub_genre_metadata.csv')

###Original datasets
df_songs = pd.read_sql("SELECT * FROM songs", con=conn_addf_metadata)
df_lyric = pd.read_csv('../Capestone/df_lyric_metadata.csv')
df_lyric_original = pd.read_sql("SELECT * FROM lyrics", con=conn_lyric)
df_genre_wide = pd.read_csv('../Capestone/df_genre_wide.csv') #key is 'track_id'
df_genre = pd.read_csv('../../Capstone/genre.csv')
#Original sub-genre/tag datasets
df_mbtag = pd.read_csv('../Capestone/df_term_artist_mbtag_wide.csv') #key is 'artist_id'
df_term = pd.read_csv('../Capestone/df_term_artist_term_wide.csv') #key is 'artist_id'
#Combined sub-genre/tag dataset
df_sub_genre = pd.read_csv('../Capestone/df_sub_genre.csv') #key is 'artist_id'

In [34]:
####Additional Cleaning

df_genre_wide = df_genre_wide.drop('Unnamed: 0', axis=1)
df_genre['genre_value'] = df_genre['genre_A'].map(lambda x: 1 if type(x) == str else 0)



In [35]:
###Removing any non ascii characters/features from the dataframe
sub_genre_del_columns = del_cols(df_sub_genre)
term_del_columns = del_cols(df_term)
mbtag_del_columns = del_cols(df_mbtag)
tags_combo_columns = del_cols(df_tags_comb)

In [36]:
###Drop the columns that don't have ascii characters
df_sub_genre = df_sub_genre.drop(sub_genre_del_columns, axis=1)
df_term =  df_term.drop(term_del_columns, axis=1)
df_mbtag =  df_mbtag.drop(mbtag_del_columns, axis=1)
df_tags_comb = df_tags_comb.drop(tags_combo_columns, axis=1)

In [37]:
df_tags_comb.shape

(3710, 218)

In [38]:
df_tags_comb = df_tags_comb[(df_tags_comb['year'] != 0)]

In [39]:
df_tags_comb.shape

(2562, 218)

In [ ]:
###Familiarity Logistic Regressionb

In [40]:
remove_cols = ['track_id', 'title', 'song_id', 'release', 'artist_id', 'artist_mbid', 
              'artist_name', 'artist_hotttnesss']

In [41]:
###remove columns for modleing
df_tags_comb_fam = df_tags_comb.drop(remove_cols, axis=1)

In [42]:
###Defining what is populuar or not
df_tags_comb_fam['artist_familiarity'].describe()

count    2562.000000
mean        0.693674
std         0.123950
min         0.023504
25%         0.609124
50%         0.693009
75%         0.791762
max         1.000000
Name: artist_familiarity, dtype: float64

In [43]:
###Categorizing anything with a score above .6 as popular
df_tags_comb_fam['familiarity'] = df_tags_comb_fam['artist_familiarity'].map(lambda x: 1 if x >= .82 else 0)

In [44]:
##Drop the 'artist_hottnesss' data
df_tags_comb_fam = df_tags_comb_fam.drop('artist_familiarity', axis=1)

In [45]:
df_tags_comb_fam.head()

,duration,year,mbtag_00s,mbtag_60s,mbtag_acid_rock,mbtag_adult_contemporary,mbtag_alternative_dance,mbtag_alternative_hip-hop,mbtag_argentina,mbtag_art_punk,...,term_technical,term_traditional_folk,term_traditional_pop,term_twee_pop,term_uk,term_virtuoso,term_vocal_house,term_western,term_zouk,familiarity
0,280.21506,2001,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1,262.89587,2004,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
2,654.28853,2009,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
3,233.58649,2001,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
4,197.19791,2000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [46]:
###Move the target column to the front of the table
cols = df_tags_comb_fam.columns.tolist()
cols = cols[-1:] + cols[:-1]
df_tags_comb_fam = df_tags_comb_fam[cols]

In [47]:
df_tags_comb_fam.head()

,familiarity,duration,year,mbtag_00s,mbtag_60s,mbtag_acid_rock,mbtag_adult_contemporary,mbtag_alternative_dance,mbtag_alternative_hip-hop,mbtag_argentina,...,term_taiwan,term_technical,term_traditional_folk,term_traditional_pop,term_twee_pop,term_uk,term_virtuoso,term_vocal_house,term_western,term_zouk
0,0,280.21506,2001,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,0,262.89587,2004,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
2,0,654.28853,2009,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
3,0,233.58649,2001,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
4,0,197.19791,2000,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [48]:
###Start to model the data frame for familiarity based on a linear regression
y = df_tags_comb_fam['familiarity'].values
X = df_tags_comb_fam.iloc[:,1:]

In [26]:
X.columns

Index([u'duration', u'year', u'mbtag_00s', u'mbtag_60s', u'mbtag_acid_rock',
       u'mbtag_adult_contemporary', u'mbtag_alternative_dance',
       u'mbtag_alternative_hip-hop', u'mbtag_argentina', u'mbtag_art_punk',
       ...
       u'term_taiwan', u'term_technical', u'term_traditional_folk',
       u'term_traditional_pop', u'term_twee_pop', u'term_uk', u'term_virtuoso',
       u'term_vocal_house', u'term_western', u'term_zouk'],
      dtype='object', length=209)

In [50]:
X.head(1)

,duration,year,mbtag_00s,mbtag_60s,mbtag_acid_rock,mbtag_adult_contemporary,mbtag_alternative_dance,mbtag_alternative_hip-hop,mbtag_argentina,mbtag_art_punk,...,term_taiwan,term_technical,term_traditional_folk,term_traditional_pop,term_twee_pop,term_uk,term_virtuoso,term_vocal_house,term_western,term_zouk
0,280.21506,2001,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0


In [51]:
ss = StandardScaler()
lrcv = LogisticRegressionCV(Cs=100, solver='liblinear', penalty='l1', cv=5)

In [52]:
Xn = ss.fit_transform(X)

In [53]:
X_train, X_test, y_train, y_test = train_test_split(Xn, y, test_size=0.33)

In [54]:
#Fit the logestic model based on the parameters - for familiarity
lrcv.fit(X_train, y_train)

LogisticRegressionCV(Cs=100, class_weight=None, cv=5, dual=False,
           fit_intercept=True, intercept_scaling=1.0, max_iter=100,
           multi_class='ovr', n_jobs=1, penalty='l1', random_state=None,
           refit=True, scoring=None, solver='liblinear', tol=0.0001,
           verbose=0)

In [55]:
print 1-np.mean(y_test)
print lrcv.score(X_test, y_test)

0.809692671395
0.886524822695


In [56]:
coefs = pd.DataFrame({'coef':lrcv.coef_[0], 'abs_coef':np.abs(lrcv.coef_[0]),
                      'variable':X.columns})
coefs.sort_values('coef', ascending=False, inplace=True)

In [57]:
coefs.shape

(209, 3)

In [58]:
coefs.head(10)

,abs_coef,coef,variable
90,1.185276,1.185276,mbtag_post-grunge
67,0.938767,0.938767,mbtag_industrial
84,0.834958,0.834958,mbtag_nu_metal
1,0.658778,0.658778,year
71,0.621373,0.621373,mbtag_irish
87,0.508281,0.508281,mbtag_parlophone
56,0.508033,0.508033,mbtag_glam_metal
22,0.507361,0.507361,mbtag_chanteur
57,0.470584,0.470584,mbtag_glam_rock
93,0.422565,0.422565,mbtag_power_pop


In [59]:
coefs.tail(10)

,abs_coef,coef,variable
142,0.475796,-0.475796,term_electric
169,0.485519,-0.485519,term_movie_score_composer
7,0.571223,-0.571223,mbtag_alternative_hip-hop
167,0.608747,-0.608747,term_melodic_rock
203,0.669434,-0.669434,term_twee_pop
52,0.671602,-0.671602,mbtag_francophone
96,0.752293,-0.752293,mbtag_protopunk
159,0.777513,-0.777513,term_jam_band
11,1.043854,-1.043854,mbtag_australian
41,1.116096,-1.116096,mbtag_electro
